In [11]:
%load_ext autoreload
%autoreload 2

from calibrated_response.models.variable import (Variable, BinaryVariable, ContinuousVariable)
from calibrated_response.maxent.constraints import (
    ConstraintSet, ProbabilityConstraint, MeanConstraint, QuantileConstraint
)
# Create a list of three variables: a binary variable, a continuous variable, and another binary variable
variables = [
    BinaryVariable(
        name="market_crash",
        description="Whether a major market crash occurs",
        importance=0.8,
        prior_probability=0.15,
        yes_label="crash",
        no_label="no crash",
        is_target=False
    ),
    ContinuousVariable(
        name="stock_return",
        description="Annual stock market return percentage",
        importance=0.9,
        lower_bound=-50,
        upper_bound=100,
        unit="percent",
        is_target=False
    ),
    BinaryVariable(
        name="recession",
        description="Whether the economy enters a recession",
        importance=0.7,
        prior_probability=0.20,
        yes_label="recession",
        no_label="no recession",
        is_target=False
    ),
]

In [8]:
# Create a list of five constraints by explicitly constructing constraint objects
constraint_var_ind = [1, 1, 1, 0, 2]
constraint_variables = [variables[i] for i in constraint_var_ind]
constraints = [
    # Constraint 1: Mean stock return around 8%
    MeanConstraint(
        id="mean_return",
        mean=8.0,
        confidence=0.85,
        source_query_id="q1"
    ),
    
    # Constraint 2: P(return > 0) = 0.70 (70% chance of positive return)
    ProbabilityConstraint(
        id="positive_return",
        lower_bound=0,
        upper_bound=100,  # domain max
        probability=0.70,
        confidence=0.9,
        source_query_id="q2"
    ),
    # Constraint 3: Median return around 10%
    QuantileConstraint(
        id="median_return",
        quantile=0.5,
        value=10.0,
        confidence=0.8,
        source_query_id="q3"
    ),
    MeanConstraint(
        id='crash_likelihood',
        mean=0.15,
        confidence=0.8,
        source_query_id="q4"
    ),
    MeanConstraint(
        id='recession_likelihood',
        mean=0.10,
        confidence=0.75,
        source_query_id="q5"
    )
]

print(f"Created {len(constraints)} constraints:")
for c in constraints:
    print(f"  - {c.id} ({c.constraint_type.value}): target={c.target_value()}, confidence={c.confidence}")

Created 5 constraints:
  - mean_return (mean): target=8.0, confidence=0.85
  - positive_return (probability): target=0.7, confidence=0.9
  - median_return (quantile): target=0.5, confidence=0.8
  - crash_likelihood (mean): target=0.15, confidence=0.8
  - recession_likelihood (mean): target=0.1, confidence=0.75


In [9]:
cs = ConstraintSet(variables=variables, constraints=constraints, constraint_variables=constraint_variables)

In [20]:
from calibrated_response.maxent.solver import MaxEntSolver
solver = MaxEntSolver()
bin_edges_list, p0 = solver.todiscrete(cs)

In [21]:
print(p0.shape)

(2, 5, 2)


In [23]:
import numpy as np

constraint_weight = 1e0

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def objective(log_p: np.ndarray) -> float:
    """Objective function: negative entropy + constraint penalties."""
    # Use log parameterization for positivity
    p = np.exp(log_p)
    p = p / p.sum()  # Normalize
    
    # Entropy (we want to maximize, so negate)
    entropy = -np.sum(p * np.log(p + 1e-12))
    neg_entropy = -entropy
    
    # Constraint violations
    penalty = 0.0
    for var_ind, c in zip(constraint_var_ind, constraints):
        p_template = ''.join(alphabet[i] for i in range(len(variables)))
        sum_template = alphabet[var_ind]
        einsum_template = f'{p_template}->{sum_template}'
        print(einsum_template, p.shape)
        var_marginal_p = np.einsum(einsum_template, p)
        bins = bin_edges_list[var_ind]
        actual = c.evaluate(var_marginal_p, bins)
        target = c.target_value()
        violation = (actual - target) ** 2
        penalty += c.confidence * violation
    
    return neg_entropy + constraint_weight * penalty

err = objective(np.log(p0))

abc->b (2, 5, 2)
abc->b (2, 5, 2)
abc->b (2, 5, 2)
abc->a (2, 5, 2)
abc->c (2, 5, 2)


In [24]:
print(err)

242.688767726466


In [13]:
print(bins, p0)

[array([-1,  0,  1]), array([-50., -20.,  10.,  40.,  70., 100.]), array([-1,  0,  1])] [0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05]
